In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from keras.models import Sequential
from keras.layers import Dense

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import pickle

from pandas.core import datetools
%matplotlib inline


Using TensorFlow backend.


# Data Acquisition
The following code imports and validates the LendingClub data.

In [2]:
# Imports loan data

df = pd.DataFrame()
basepath='./Source Data/Loan Data/'
files = os.listdir(basepath)
csvs = []

for file in files:
    if re.match('.*csv$',file):
        csvs += [file]

try:
    #ignore this - was trying to pickle the data into
    #formats like feather, hdf5, native python pickling, etc
    # but found issues on python 3.7
    df=pickle.load(open('full_data.p', 'rb'))
    print('Read from pickle')
except:
    dates = [
        'next_pymnt_d',
        'hardship_start_date',
        'hardship_end_date',
        'payment_plan_start_date',
        'earliest_cr_line',
        'issue_d',
        'debt_settlement_flag_date',
        'last_pymnt_d'

    ]
    cols = df.dtypes
    for csv in csvs:
        path = basepath + csv
        print("Reading",path)
        tdf = pd.read_csv(path,header=1,parse_dates=dates,low_memory=False)
        df=df.append(tdf)
    df.reset_index(inplace=True) # This will help with joining back data if necessary.
    # Convert dates to datetime
    #df['issue_d'] = pd.to_datetime(df['issue_d'])
    #df['debt_settlement_flag_date'] = pd.to_datetime(df['debt_settlement_flag_date'])
    df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
    
    #determine age of credit line prior to loan issue and convert to integer
    # days of credit history
    df['earliest_cr_line'] = (df['issue_d']-df['earliest_cr_line']).dt.days
    
    # convert issue_d to a year to consider economic conditions
    #SHOULD WE GO TO QUARTERS?
    #df['issue_d'] = df['issue_d'].dt.year
    
    df['duration'] = (df['last_pymnt_d'] - df['issue_d']).dt.days / 30
    pickle.dump( df, open( "full_data.p", "wb" ) )

print(df.shape)

Read from pickle
(1873317, 153)


In [3]:
# Limit to loans that are paid in full or written off. Uses dates so that 
# loans that are delinquent are not disproportionaltely dropped from data

mature_filter = (df['loan_status']=='Fully Paid')|(df['loan_status']=='Charged Off')
reduced_df = df[mature_filter] # Pulls only loans that are charged off or paid in full.
#
## Use my documentation to filter to only useful regressors
data_dict = pd.read_excel('./Source Data/LCDataDictionary.xlsx',sheet_name='LoanStats')
features = list(data_dict[data_dict['Useful Predictor']=='Yes']['LoanStatNew'].values)+['duration']
reduced_df=reduced_df[features]

# Combines fields when necessary
reduced_df['fico_est'] = (reduced_df['fico_range_low']+reduced_df['fico_range_high'])/2

reduced_df.drop(columns=['fico_range_low','fico_range_high'],inplace=True)
print(reduced_df.shape)

(1020552, 70)


In [4]:
# Convert strings to numbers emp_length, int_rate, revol_util
emp_length_map={'10+ years':10, '< 1 year':0, '1 year':1, '3 years':3, '8 years':8, '9 years':9,
                '4 years':4, '5 years':5, '6 years':6, '2 years':2, '7 years':7}

reduced_df['emp_length']=reduced_df['emp_length'].replace(pd.Series(emp_length_map))

grade_map={"A":1,"B":2,"C":3,"D":4,"E":5,"F":6,"G":7}
reduced_df['grade']=reduced_df['grade'].replace(pd.Series(grade_map))

reduced_df['int_rate']=reduced_df['int_rate'].apply(lambda x: float(x[:-1]))
reduced_df['revol_util']=reduced_df['revol_util'].apply(lambda x:
                                                        x[:-1] if isinstance(x, str) else np.nan).astype(float)

reduced_df['earliest_cr_line']=reduced_df['earliest_cr_line'].apply(lambda x:
                                                        0.0 if np.isnan(x) else x)

reduced_df.drop(columns=['emp_title'],inplace=True)

print(reduced_df.shape)

(1020552, 69)


In [5]:
seta=set(reduced_df.columns)


# 8/8 consolidated purpose
# Consolidated where logical
reduced_df['purpose'].replace('credit_card','debt_consolidation',inplace=True)
reduced_df['purpose'].replace('educational','other',inplace=True)
reduced_df['purpose'].replace('wedding','major_purchase',inplace=True)
reduced_df['purpose'].replace('vacation','major_purchase',inplace=True)
reduced_df['purpose'].replace('moving','house',inplace=True)
reduced_df['purpose'].replace('home_improvement ','house',inplace=True)
reduced_df['purpose'].replace('renewable_energy ','house',inplace=True)

reduced_df=pd.get_dummies(data=reduced_df,columns=['application_type','term',
                                                   'verification_status','purpose'],
                          drop_first=True)

# 8/8 removed 'home_ownership
reduced_df['home_ownership'] = np.where(reduced_df['home_ownership']=='OWN',1,0)

setb=set(reduced_df.columns)
print(setb-seta)
print(reduced_df.shape)

{'application_type_Joint App', 'verification_status_Verified', 'purpose_major_purchase', 'purpose_other', 'purpose_medical', 'verification_status_Source Verified', 'purpose_house', 'term_ 60 months', 'purpose_home_improvement', 'purpose_renewable_energy', 'purpose_debt_consolidation', 'purpose_small_business'}
(1020552, 77)


# How to treat NaN?

For now, let's remove majority NaN columns...

In [6]:
has_data = {}
for column in reduced_df.columns:
    has_data[column] = len(reduced_df[column].dropna())
has_data

order_has_data=sorted(has_data, key=lambda dict_key: has_data[dict_key])

top_sparse=25
for i,j in zip(range(top_sparse),order_has_data[0:top_sparse]):
    print(i,j, has_data[j])

0 annual_inc_joint 9529
1 dti_joint 9529
2 mths_since_last_record 172185
3 mths_since_recent_bc_dlq 240759
4 mths_since_last_major_derog 264308
5 mths_since_rcnt_il 289098
6 all_util 296067
7 inq_last_12m 296092
8 open_acc_6m 296092
9 total_cu_tl 296092
10 inq_fi 296093
11 open_il_12m 296093
12 open_il_24m 296093
13 open_rv_12m 296093
14 open_rv_24m 296093
15 mths_since_recent_revol_delinq 338379
16 mths_since_last_delinq 503506
17 mths_since_recent_inq 881746
18 mo_sin_old_il_acct 924972
19 mo_sin_old_rev_tl_op 953024
20 mo_sin_rcnt_rev_tl_op 953024
21 num_rev_accts 953024
22 mo_sin_rcnt_tl 953025
23 num_accts_ever_120_pd 953025
24 num_bc_tl 953025


In [7]:
nonnan_df=reduced_df.drop(columns=order_has_data[0:19]).dropna()
nonnan_df = nonnan_df[nonnan_df['duration']!=0]
print(nonnan_df.shape)

(882558, 58)


In [8]:
reduced_df['duration'].describe()

count    1.018723e+06
mean     2.010069e+01
std      1.230952e+01
min     -0.000000e+00
25%      1.013333e+01
50%      1.830000e+01
75%      3.043333e+01
max      7.100000e+01
Name: duration, dtype: float64

# Interaction Variables
I added anything that an amount as an interaction variable compared with income.

In [9]:
poly_vars = ['dti','inq_last_6mths']

for col in poly_vars:
    new_title = col + '_2'
    nonnan_df[new_title] = nonnan_df[col] * nonnan_df[col]
    new_title = col + '_3'
    nonnan_df[new_title] = nonnan_df[col] * nonnan_df[col] * nonnan_df[col]
for col in ['revol_util','tot_coll_amt','loan_amnt','installment']:
    new_title = col + '_over_inc'
    nonnan_df[new_title] = nonnan_df[col] / nonnan_df['annual_inc']

# Let's work with training and test sets

In [ ]:
print(df.shape,nonnan_df.shape)

# need to look at interest rate as well!!!
y=nonnan_df['total_pymnt']/(nonnan_df['duration'])

traintest_df=nonnan_df.drop(columns=['total_pymnt','duration'])
traintest_df['issue_d'] = traintest_df['issue_d'].dt.year

X_train, X_test, y_train, y_test = train_test_split(
    traintest_df,y,test_size=0.2,random_state=42)#,stratify=nonnan_df[['loan_status']])

Xscaler = StandardScaler()
Xscaler.fit_transform(X_train,X_test)

print(X_train.shape,X_test.shape)

In [ ]:
LCOLSModel = LinearRegression().fit(X_train, y_train)

print(LCOLSModel.intercept_)
print(LCOLSModel.coef_)

# your code here
trainR2 = r2_score(y_train,LCOLSModel.predict(X_train))

testR2 = r2_score(y_test,LCOLSModel.predict(X_test))

print("The training set OLS regression R^2 score is: %f" % trainR2)
print("The test set OLS regression R^2 score is: %f" % testR2)

In [ ]:
lambdas = [.001, .005, 1, 5, 10, 50, 100, 500, 1000]

kfold = KFold(5, shuffle=True) # use this for testing

LCRRModel = RidgeCV(alphas=lambdas, cv=kfold)
LCRRModel.fit(X_train, y_train)
LCRR_shrinkage_parameter=LCRRModel.alpha_

print("Best model searched:\nalpha = {}\nintercept = {}\nbetas = {}, ".format(LCRR_shrinkage_parameter,
                                                                            LCRRModel.intercept_,
                                                                            LCRRModel.coef_
                                                                            )
     )

RRtrainR2 = r2_score(y_train,LCRRModel.predict(X_train))
RRtestR2 = r2_score(y_test,LCRRModel.predict(X_test))
print("The training set Ridge regression R^2 score is: %f" % RRtrainR2)
print("The test set Ridge regression R^2 score is: %f" % RRtestR2)

In [ ]:
LCLRModel = LassoCV(alphas=lambdas, cv=kfold)
LCLRModel.fit(X_train, y_train)
LCLR_shrinkage_parameter = LCLRModel.alpha_
print("Best model searched:\nalpha = {}\nintercept = {}\nbetas = {}, ".format(LCLR_shrinkage_parameter,
                                                                            LCLRModel.intercept_,
                                                                            LCLRModel.coef_
                                                                            )
     )

LRtrainR2 = r2_score(y_train,LCLRModel.predict(X_train))
LRtestR2 = r2_score(y_test,LCLRModel.predict(X_test))
print("The training set Lasso regression R^2 score is: %f" % LRtrainR2)
print("The test set Lasso regression R^2 score is: %f" % LRtestR2)

In [ ]:
coef_df=pd.DataFrame(np.array([X_train.columns,
                               LCOLSModel.coef_,
                               LCRRModel.coef_,
                               LCLRModel.coef_]).T,columns=["feature","OLS","RR","LR"])

In [ ]:
print(coef_df)

In [ ]:
# your code here
plt.figure(figsize=(20,10))
plt.bar(coef_df["feature"],[-abs(x) for x in coef_df["OLS"].values],label='OLS',alpha=1.0)
plt.bar(coef_df["feature"],-abs(coef_df["RR"]),label='RR',alpha=0.5)
plt.bar(coef_df["feature"],abs(coef_df["LR"]),label='LR',alpha=0.5)
plt.yscale("linear")

plt.legend()
plt.show()

In [ ]:
coef_df[abs(coef_df['OLS'])>100]

# Advanced Models
We continue to have problems with multicolinearity which may be contributing to some of the problems we are seeing in the data. I am going to try additional models that may be more suitable.

Because of the sheer size of our dataset, the following block of code take a long time to run. I ran one with full datasets without much luck. To reduce the runtime, I pulled a random sample from the dataset.

In [10]:
sample_percent = .05 # I put this ridiculouly low to ensure code would compile. I will run again overnight with a 
# larger number for additional insights.
X_train_sample = X_train.reset_index(drop=True).sample(int(len(X_train)*sample_percent),random_state=42)
y_train_sample = y_train.reset_index(drop=True).iloc[X_train_sample.index]

NameError: name 'X_train' is not defined

In [ ]:
neu_net = Sequential([
    Dense(800, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(800, activation='relu'),
    Dense(400, activation='relu'),
    Dense(400, activation='relu'),
    Dense(400, activation='relu'),
    Dense(1, activation='relu')
])
neu_net.compile(loss='mean_absolute_error', optimizer='adam')
neu_net.summary()

In [ ]:
#neu_net.fit(X_train_sample,y_train_sample, epochs=20, batch_size=64, validation_split = .25)
# I was not seeing improvement between epochs. This is not sutiable data for neural net

I don't see an improvement in the loss function. I'm going to try alternative methods.

In [ ]:
RF_model = RandomForestRegressor(10)
RF_model.fit(X_train_sample,y_train_sample)
RF_model.score(X_test,y_test)

In [ ]:
size = []
avg_score = []
sd_score = []
for i in range(5,30):
    s = i * 4
    size += [s]
    RF_model = RandomForestRegressor(s)
    cv = cross_val_score(RF_model,X_train_sample,y_train_sample,cv=6,n_jobs=4)
    avg_score +=  [np.average(cv)]
    sd_score += [np.std(cv)]
    print(i,'complete')
    print(avg_score,'avg_score')
    

In [ ]:
plt.plot(size,avg_score)
bound = []
for i in range(len(size)):
    bound += [avg_score[i]+sd_score[i]]
plt.plot(size,bound,c='orange')

bound = []
for i in range(len(size)):
    bound += [avg_score[i]-sd_score[i]]
plt.plot(size,bound,c='orange')